In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import cvxpy as cp

from cartpole.cartpole import Cartpole
import pickle, os

In [2]:
#load train/test data
prob = Cartpole() #use default config, pass different config file oth.
config_fn = './cartpole/config/default.p'

config_file = open(config_fn,'rb')
dataset_name, _, _ = pickle.load(config_file); config_file.close()

relative_path = os.getcwd()
dataset_fn = relative_path + '/cartpole/data/' + dataset_name

train_file = open(dataset_fn+'/train.p','rb')
# p_train, x_train, u_train, y_train, c_train, times_train = pickle.load(train_file)
train_data = pickle.load(train_file)
train_file.close()

test_file = open(dataset_fn+'/test.p','rb')
# p_test, x_test, u_test, y_test, c_test, times_test = pickle.load(test_file)
test_data = pickle.load(test_file)
test_file.close()

In [ ]:
#pre-processing

#some function here to convert p_test into a feature matrix, X
#some function here to convert y_test into strategy labels, y

In [ ]:
#training setup

#instantiate Solver object
#if first time, setup hyperparams + write out to a config file
#else, load hyperparams from config (e.g. network weights, lr, etc.)

In [ ]:
#training step

#either we can write a big loop here, or hide it in some Solver.train() method.

In [ ]:
#testing setp

#either we can write a big loop here, or hide it in some Solver.test() method.

In [ ]:
## dummy cell to test basic Cartpole functionality

xg = 0.1*np.random.randn(4); x0 = 0.1*np.random.randn(4) #dummy param values
params = {'x0': x0,'xg' : xg} #put them into a dict
strat = np.zeros((4,prob.N-1)).astype(int) #dummy strategy (all zeros)

feas_bin, optval_bin, time_bin, assigns_bin = prob.solve_micp(params) #solve problem exactly
feas, optval, time = prob.solve_pinned(params,strat)

# test MLOPT solver

In [3]:
from solvers.mlopt import MLOPT

system = 'cartpole'
prob_features = ['x0', 'xg']
mlopt_obj = MLOPT(system, prob, prob_features)

n_features = 8
mlopt_obj.construct_strategies(n_features, train_data)
print(mlopt_obj.n_strategies)

589


In [22]:
mlopt_obj.setup_network()

fn_saved = 'mlopt_cartpole_20200622_1815.pt'
mlopt_obj.load_network(fn_saved)

mlopt_obj.model

Loading presaved classifier model from mlopt_cartpole_20200622_1815.pt


FFNet(
  (activation): ReLU()
  (layers): ModuleList(
    (0): Linear(in_features=8, out_features=32, bias=True)
    (1): Linear(in_features=32, out_features=32, bias=True)
    (2): Linear(in_features=32, out_features=32, bias=True)
    (3): Linear(in_features=32, out_features=589, bias=True)
  )
)

In [ ]:
mlopt_obj.training_params['TRAINING_ITERATIONS'] = 1000
mlopt_obj.train()
print(mlopt_obj.model_fn)

In [24]:
n_succ = 0
count = 0

for ii in range(1000):
    prob_params = {}
    idx = np.random.randint(test_data[1].shape[0])

    p_test = test_data[0]
    for k in p_test.keys():
        prob_params[k] = p_test[k][idx]
    prob_success, cost, total_time, n_evals = mlopt_obj.forward(prob_params)

    count += 1
    if prob_success:
        n_succ += 1

print(float(n_succ)/float(count))

1.0


In [25]:
prob_params = {}
idx = np.random.randint(test_data[1].shape[0])

p_test = test_data[0]
for k in p_test.keys():
    prob_params[k] = p_test[k][idx]
prob_params

{'x0': array([ 0.46468615, -0.27337426, -0.85975189,  0.40075829]),
 'xg': array([0., 0., 0., 0.])}

In [27]:
prob_success, cost, total_time, n_evals = mlopt_obj.forward(prob_params)

if prob_success:
    print(cost)
else:
    print('Failed')

144.26570218216074


In [28]:
prob_success, true_cost, solve_time, optvals = mlopt_obj.problem.solve_micp(prob_params)

true_cost

Academic license - for non-commercial use only


144.2656965961387

In [29]:
n_succ = 0
count = 0

for ii in range(100):
    prob_params = {}
    idx = np.random.randint(test_data[1].shape[0])

    p_test = test_data[0]
    for k in p_test.keys():
        prob_params[k] = p_test[k][idx]
    prob_success, cost, total_time, n_evals = mlopt_obj.forward(prob_params)

    count += 1
    if prob_success:
        n_succ += 1

print(float(n_succ)/float(count))

1.0


# test Regression solver

In [ ]:
from solvers.regression import Regression

system = 'cartpole'
prob_features = ['x0', 'xg']
reg_obj = Regression(system, prob, prob_features)

n_features = 8
reg_obj.construct_strategies(n_features, train_data)
print(reg_obj.n_strategies)

In [ ]:
reg_obj.setup_network()
reg_obj.model

In [ ]:
reg_obj.training_params['TRAINING_ITERATIONS'] = 10
reg_obj.training_params['CHECKPOINT_AFTER'] = 200000
reg_obj.train()
print(reg_obj.model_fn)

In [ ]:
prob_params = {}
idx = 1

p_test = train_data[0]
for k in p_test.keys():
    prob_params[k] = p_test[k][idx]
prob_params

In [ ]:
prob_success, cost, total_time = reg_obj.forward(prob_params)

if prob_success:
    print(cost)
else:
    print('Failed')